#  Display options:

In [ ]:

#  References for this Notebook,
#
#     Working thru the examples on this page,
#        https://docs.k9h.dev/latest/recipes/udf-recipes.html/tree/master/lonestar/analytics/distributed/experimental
# 


In [ ]:

#  Setting display options 

import pandas as pd
   #
pd.set_option("display.width", 480)

#  Sets horizontal scroll for wide outputs
#
from IPython.display import display, HTML
display(HTML(""))

from tabulate import tabulate

print("--")


#  Setup stuff: Connectivity

In [ ]:

from katana import remote
#  from katana.remote import import_data

my_client = remote.Client()

print(my_client)


In [ ]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


In [ ]:

#  CONNECT TO GRAPH

my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)


In [ ]:

display(my_graph.num_nodes())
display(my_graph.num_edges())


# Working with num_partitions()

In [ ]:
#  Even though this graph was made with 3 partitions, and even though certain operations
#  require at least 3 partitions else they fail, you may still see num_partitions = None
#  which can happen on really small graphs, and produce unexpected results below.

print(my_graph.num_partitions)


In [ ]:

#  my_graph.repartition(num_partitions = 1)
my_graph.repartition(num_partitions = 3)

print("--")


# UDFs, Example 01: Count City Nodes

In [ ]:

#  All of these below are translated to the Airport graph
#

#  Not an exact copy of example 1, some things left off-
#
def my_func(i_graph, i_property):
    
   l_airport_seq = i_graph.nodes(labels = i_property)
      #
   print("Type: %s" % (str(type(l_airport_seq))))
   print("Value: %s" % (l_airport_seq))
   print("Loop thru the above:")
      #
   for l_each in l_airport_seq:
      print("   " + str(l_each))
        
   print("Airport Node Count", len(l_airport_seq))

   return

l_result = my_graph.run(lambda g: my_func(g, "Airport"))


#  Sample output,
#
#     Host 0 output:
#     Type: <class 'katana.distributed.EntitySequenceNode'>
#     Value: <katana.distributed.EntitySequenceNode object at 0x7f8da9fbaf30>
#     Loop thru the above:
#        <lnid 4294967294>
#        <lnid 0>
#        <lnid 1>
#     Airport Node Count 3
#     
#     Host 1 output:
#     Type: <class 'katana.distributed.EntitySequenceNode'>
#     Value: <katana.distributed.EntitySequenceNode object at 0x7f90c5ab6df0>
#     Loop thru the above:
#        <lnid 4294967293>
#        <lnid 4294967294>
#        <lnid 0>
#     Airport Node Count 3
#     
#     Host 2 output:
#     Type: <class 'katana.distributed.EntitySequenceNode'>
#     Value: <katana.distributed.EntitySequenceNode object at 0x7fe5882e4230>
#     Loop thru the above:
#        <lnid 4294967293>
#        <lnid 4294967294>
#        <lnid 0>
#     Airport Node Count 3


In [ ]:

#  All of these below are translated to the Airport graph
#

#  An exact copy of example 1
#
def my_func(i_graph, i_property):
    
   l_airport_seq: Sequence[Node] = i_graph.nodes(labels = i_property)
      #
   print("Type: %s" % (str(type(l_airport_seq))))
   print("Value: %s" % (l_airport_seq))
   print("Loop thru the above:")
      #
   for l_each in l_airport_seq:
      print("   " + str(l_each))

   print("Airport Node Count", len(l_airport_seq))

   return

l_result = my_graph.run(lambda g: my_func(g, "Airport"))


#  Sample output,
#
#     Host 0 output:
#     Type: <class 'katana.distributed.EntitySequenceNode'>
#     Value: <katana.distributed.EntitySequenceNode object at 0x7fe27927b7f0>
#     Loop thru the above:
#        <lnid 4294967294>
#        <lnid 0>
#        <lnid 1>
#     Airport Node Count 3
#     
#     Host 1 output:
#     Type: <class 'katana.distributed.EntitySequenceNode'>
#     Value: <katana.distributed.EntitySequenceNode object at 0x7fc0f80d80f0>
#     Loop thru the above:
#        <lnid 4294967293>
#        <lnid 4294967294>
#        <lnid 0>
#     Airport Node Count 3
#     
#     Host 2 output:
#     Type: <class 'katana.distributed.EntitySequenceNode'>
#     Value: <katana.distributed.EntitySequenceNode object at 0x7f96742bcc30>
#     Loop thru the above:
#        <lnid 4294967293>
#        <lnid 4294967294>
#        <lnid 0>
#     Airport Node Count 3


# UDFs, Example 02: Source Node Labels

In [ ]:

#  Example 02, mostly as written, converted to airports
#
#     .  We add a second for loop for to also process out edges.
#        This graph had one out edge, from Airport to Restaurant
#

def my_func(i_graph, i_property):
    
   l_airport_seq: Sequence[Node] = i_graph.nodes(labels = i_property)

   l_labels = set()                                                       #  Python sets discard duplicates
 
   for l_node in l_airport_seq:                                           #  All nodes of a given label
      l_each: Node
      for l_edge in i_graph.in_edges(l_node):                             #  In edges
         l_edge: InEdge
         l_src_node: Node = i_graph.in_edges.src(l_edge)
         l_labels.add(i_graph.nodes.labels(l_src_node))
        
      for l_edge in i_graph.out_edges(l_node):                            #  Out edges
         l_edge: OutEdge
         l_dst_node: Node = i_graph.out_edges.dst(l_edge)
         l_labels.add(i_graph.nodes.labels(l_dst_node))
           
   print(l_labels)
    

   return

l_result = my_graph.run(lambda g: my_func(g, "Airport"))

#  Sample output,
#
#     Host 0 output:
#     {Airport}
#     
#     Host 1 output:
#     {Airport, Restaurant}
#     
#     Host 2 output:
#     {Airport}


# UDFs, Example 03: Out-degree of a City Nodes

In [28]:

def my_func(i_graph, i_property):
    
   l_airport_seq: Sequence[Node] = i_graph.nodes(labels = i_property)

   if (l_airport_seq):
      print("LLL")


   return

l_result = my_graph.run(lambda g: my_func(g, "Airport2"))




          0/? [?op/s]


Host 0 output:
<class 'katana.distributed.EntitySequenceNode'>

Host 1 output:
<class 'katana.distributed.EntitySequenceNode'>

Host 2 output:
<class 'katana.distributed.EntitySequenceNode'>


In [ ]:





    city_sequence: Sequence[Node] = graph.nodes(labels="City")
    
    if city_sequence:
        city, *_ = city_sequence
        # Get the global out degree of a city.
        print("out_degree:", graph.nodes.global_out_degree(city))
        # Also available for in-edges.
        
        
        

In [ ]:

def my_func(i_graph, i_property):
    

   return

l_result = my_graph.run(lambda g: my_func(g, "Airport"))
